In [8]:
## Task2
city_graph = {   #city   :  customer
    'A': [('B', 4), ('C', 3)],
    'B': [('A', 4), ('D', 5), ('E', 2)],
    'C': [('A', 3), ('E', 6)],
    'D': [('B', 5), ('F', 3)],
    'E': [('B', 2), ('C', 6), ('F', 8)],
    'F': [('D', 3), ('E', 8)] # Goal State
}

initial_state = 'A'  # Depot
goal_state = 'D'     # Customer

#Checks if current state equals customer location."""
def goal_test(state):
    return state == goal_state

def get_actions(state):
    """Possible Actions: MoveTo(Location)."""
    return city_graph.get(state, [])


#Transition Model: Returns the new state.
def result(action):
    neighbor, cost = action
    return neighbor

#Checks all reachable states from the initial state.
def generate_state_space(graph, start):
    visited = set()
    stack = [start]
    while stack:
        current = stack.pop()
        if current not in visited:
            visited.add(current)
            for neighbor, cost in graph.get(current, []):
                stack.append(neighbor)
    return list(visited)


def search_solution(current_state, goal, path=None, total_cost=0):
    if path is None:
        path = [current_state]
    
    if current_state == goal:
        return [(path, total_cost)]
    
    solutions = []
    for action in get_actions(current_state):
        neighbor = result(action)
        cost = action[1]
        
       # 3. stops cycle like if we r going to same node again
        if neighbor not in path:
            new_path = path + [neighbor]
            new_cost = total_cost + cost
            results = search_solution(neighbor, goal, new_path, new_cost)
            solutions.extend(results)
            
    return solutions


print("Route Planning Agent Results ")
print(f"Initial State: {initial_state}")
print(f"Goal State: {goal_state}\n")


reachable = generate_state_space(city_graph, initial_state)
print(f"State-space graph (Reachable States): {reachable}\n")

# Optimal Solution and Cost
all_paths = search_solution(initial_state, goal_state)
if all_paths:
   
    optimal_path, min_cost = min(all_paths, key=lambda x: x[1])
    print("Solution Action Sequence (Optimal Route):")
    print(" to ".join(optimal_path))
    print(f"Total Path Cost: {min_cost} units")

Route Planning Agent Results 
Initial State: A
Goal State: D

State-space graph (Reachable States): ['C', 'A', 'B', 'F', 'E', 'D']

Solution Action Sequence (Optimal Route):
A to B to D
Total Path Cost: 9 units


## TASK 3

In [7]:
from collections import deque


GOAL_STATE = (
    (1,2,3),
    (4,5,6),
    (7,8,0)
)

INITIAL_STATE = (
    (1,2,3),
    (4,0,6),
    (7,5,8)
)


MAX_DEPTH = 10

print("Initial State:")
for row in INITIAL_STATE:
    print(row)
    
print("\nGoal State:")

for row in GOAL_STATE:
    print(row)


# Goal test
def goal_test(state):
    return state == GOAL_STATE


def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

# Get valid moves from a state
def get_valid_moves(state):
    moves=[]
    row,col = find_blank(state)
    if row>0:
        moves.append("Up")
    if row<2:
        moves.append("Down")
    if col>0:
        moves.append("Left")
    if col<2:
        moves.append("Right")
    return moves

#returns new statee
def apply_move(state, move):
    row, col = find_blank(state)
    new_state = [list(r) for r in state]  # convert to list
    if move=="Up":
        new_state[row][col], new_state[row-1][col] =new_state[row-1][col], new_state[row][col]
    elif move=="Down":
        new_state[row][col], new_state[row+1][col] =new_state[row+1][col], new_state[row][col]
    elif move=="Left":
        new_state[row][col], new_state[row][col-1] =new_state[row][col-1], new_state[row][col]
    elif move=="Right":
        new_state[row][col], new_state[row][col+1] =new_state[row][col+1], new_state[row][col]
    return tuple(tuple(r) for r in new_state)  # convert back to tuple of tuples


    # Test functions
print("Is Starting State Goal?", goal_test(INITIAL_STATE))
print("Blank Position:", find_blank(INITIAL_STATE))
print("All Valid Moves:", get_valid_moves(INITIAL_STATE))

# Apply each move and show new state
for move in get_valid_moves(INITIAL_STATE):
    new_state = apply_move(INITIAL_STATE, move)
    print(f"\nMove: {move}")
    for row in new_state:
        print(row)


def generate_state_space(initial_state, max_depth=MAX_DEPTH):
    visited = set()  # to avoid duplicates
    state_info = dict()  # parent mapping: state -> (parent, move)

    queue = deque()
    queue.append((initial_state, 0, None, None))  # (state, depth, parent, move)
    visited.add(initial_state)
    state_info[initial_state] = (None, None)

    solution_found = False
    solution_state = None

    while queue:
        current_state, depth, parent, move = queue.popleft()

        if goal_test(current_state):
            solution_found = True
            solution_state = current_state
            break

        if depth >= max_depth:
            continue

        for m in get_valid_moves(current_state):
            new_state = apply_move(current_state, m)
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, depth+1, current_state, m))
                state_info[new_state] = (current_state, m)

    return solution_found, solution_state, state_info, len(visited)




def reconstruct_path(solution_state, state_info):
    path = []
    current = solution_state
    while state_info[current][0] is not None:
        parent, move = state_info[current]
        path.append(move)
        current = parent
    path.reverse()
    return path




solution_found, solution_state, state_info, states_generated = generate_state_space(INITIAL_STATE)

print("Number of States Generated:", states_generated)

if solution_found:
    print("\nSolution Found!")
    solution_path = reconstruct_path(solution_state, state_info)
    print("Solution Path (moves):", solution_path)
    print("Path Cost:", len(solution_path))
else:
    print("\nNo solution found within depth limit.")

Initial State:
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)

Goal State:
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
Is Starting State Goal? False
Blank Position: (1, 1)
All Valid Moves: ['Up', 'Down', 'Left', 'Right']

Move: Up
(1, 0, 3)
(4, 2, 6)
(7, 5, 8)

Move: Down
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

Move: Left
(1, 2, 3)
(0, 4, 6)
(7, 5, 8)

Move: Right
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
Number of States Generated: 16

Solution Found!
Solution Path (moves): ['Down', 'Right']
Path Cost: 2
